In [1]:
print("Test Scucessfully!")

Test Scucessfully!


# Load Dataset and Model

In [2]:
from datasets import load_dataset

# Load Dataset
dataset_name = "glue"
task_name = "sst2"
dataset = load_dataset(dataset_name, task_name, split="train")
validation = load_dataset(dataset_name, task_name, split="validation")

Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# Version 2-7b
# base_model_name = "meta-llama/Llama-2-7b-hf"
# Version 2-13b-chat
# base_model_name = "meta-llama/Llama-2-13b-chat-hf"
# Ver

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)

base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

/usr/local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
tokenizer

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-13b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False)

# Inference

### Inference Mode 1

In [6]:
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative."
user_msg_1 = "I am so happy today."

text = system_prompt + "\n" + user_msg_1 + "\n" + "Sentiment:"

inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.
I am so happy today.
Sentiment: Positive


### Inference Mode 2

In [7]:
prompt = "What is a llama?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

What is a llama?

A llama is a large, domesticated South American mammal that is closely related to the camel. It is known for its distinctive long neck, ears, and legs, as well as its soft, woolly fur. Llamas are often used as pack animals, and are also kept as pets and show animals. They are native to the Andean regions of South America, and have been bred by humans for thousands of years.



What is a llama used for?

Llamas are used for a variety of purposes, including:



1. Packing: Llamas are often used as pack animals, carrying goods and supplies over long distances.

2. Transportation: Llamas can be ridden like horses, and are sometimes used as a mode of transportation in rural areas.

3. Agriculture: Llamas are used


## Few Inference

In [8]:
# System Prompt
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative."
# User Prompt
user_prompt = "Please classify the sentiment of the following sentence into positive or negative:"
# Examples
examples = """
Here is an inference example:
Sentence: I am so happy today.
Sentiment: positive

Here is another inference example:
Sentence: I am so sad today.
Sentiment: negative

"""

In [9]:
import re
from tqdm import tqdm

def evaluate(dataset):
    label_map = {
        0 : 'negative',
        1 : 'positive',
    }

    compared_result = []

    for i, val in enumerate(tqdm(dataset)):
        label_text = label_map[val['label']]
        sentence = val['sentence']

        # Make input
        text = user_prompt + examples + "\n" + "Sentence: " + sentence + "\n" + "Sentiment: "
        inputs = tokenizer(text, return_tensors="pt").to("cuda")

        # Generate
        outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)
        # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
        outputs_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        matches = re.findall(r"Sentiment: (.+)", outputs_text)
        selected_sentiment = matches[2].strip() if len(matches) >= 3 else None
        # print(selected_sentiment)

        # Compare prediction and label
        if selected_sentiment == label_text:
            compared_result.append(1)
        else:
            compared_result.append(0)

    return compared_result


In [10]:
result_list = evaluate(validation)

100%|██████████| 872/872 [14:06<00:00,  1.03it/s]


In [11]:
print("Accuracy:", result_list.count(1)/len(result_list))

Accuracy: 0.8623853211009175
